# Deploy ESM3-open Model Package from AWS Marketplace 


--------------------

## <font color='orange'>Important:</font>

Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq">https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.

-------------------


ESM3 is a frontier generative model for biology, able to jointly reason across three fundamental biological properties of proteins: sequence, structure, and function. These three data modalities are represented as tracks of discrete tokens at the input and output of ESM3. You can present the model with a combination of partial inputs across the tracks, and ESM3 will provide output predictions for all the tracks.
ESM3 is a generative masked language model. You can prompt it with partial sequence, structure, and function keywords, and iteratively sample masked positions until all positions are unmasked.

<img src="https://aws-hcls-ml.s3.amazonaws.com/public_assets_support_materials/esm3/jumpstart/img/esm3-architecture.png" alt="ESM3 Architecture" style="width:80%;"/>


The ESM3 architecture is highly scalable due to its transformer backbone and all-to-all reasoning over discrete token sequences. At its largest scale, ESM3 was trained with 1.07e24 FLOPs on 2.78 billion proteins and 771 billion unique tokens, and has 98 billion parameters.
Here we present esm3-open-small. With 1.4B parameters it is the smallest and fastest model in the family, trained specifically to be open sourced. ESM3-open is available under a non-commercial license.

This sample notebook shows you how to deploy [EvolutionaryScale - ESM3](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

> ESM3 model package support SageMaker Realtime Inference but not SageMaker Batch Transform.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [ESM3](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#A.-Create-a-model-from-the-subscribed-model-package)
   1. [Create a model from the subscribed model package](#A.-Create-an-endpoint-configuration)
   2. [Create an endpoint configuration](#B.-Create-an-endpoint-configuration)
   3. [Create a realtime inference endpoint](#C.-Create-an-endpoint-configuration)
   4. [Create input payload](#D.-Create-an-input-payload)
   5. [Perform real-time inference](#E.-Perform-real-time-inference)   
   5. [Delete the endpoint](#H.-Delete-the-endpoint)
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [EvolutionaryScale ESM3 Model](https://aws.amazon.com/marketplace/pp/prodview-xbvra5ylcu4xq)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [5]:
!pip install --upgrade boto3 sagemaker
# if you upgrade the package, you need to restart the kernel


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
import boto3
import sagemaker
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker', region_name=region)

try:
    # Attempt to get the execution role
    role = sagemaker.get_execution_role()
    print(f"Default SageMaker execution role: {role}")
except ValueError as e:
    print(f"Error getting default execution role: {e}")
    print("You may need to specify a role explicitly or create one if not running in a SageMaker environment.")

esm3_package = "esm3-sm-open-v1-dev-9cba7454472e39b68b394d9c7c70e393"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{esm3_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{esm3_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{esm3_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{esm3_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{esm3_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{esm3_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{esm3_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{esm3_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{esm3_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{esm3_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{esm3_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{esm3_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{esm3_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{esm3_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{esm3_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{esm3_package}",
}

if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]

Default SageMaker execution role: arn:aws:iam::471112526672:role/StandardAuthorization


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Create a model from the subscribed model package

You can create a model from your SageMaker marketplace subscriptions. 

<img src="https://aws-hcls-ml.s3.amazonaws.com/public_assets_support_materials/esm3/jumpstart/img/model-from-marketplace.png" alt="Subscribed model - ESM3" style="width:80%;"/>

ESM3 model package is available to be used to create a model that can be hosted behind a realtime inference endpoint


In [2]:
# Alternatively you can also create a model package programmatically by copying the model package ARN from the subscriptions screen
from sagemaker.model import ModelPackage
print(f"Model package ARN: {model_package_arn}")
endpoint_name = sagemaker.utils.name_from_base("ESM3-open")

model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session
)

Model package ARN: arn:aws:sagemaker:us-east-2:057799348421:model-package/esm3-sm-open-v1-dev-9cba7454472e39b68b394d9c7c70e393


### B. Create an endpoint configuration

ESM3 allows inference only on `ml.g5.2xlarge` or `ml.g5.4xlarge` instance types. Let's create an endpoint configuration which supports one of these instance types

Once endpoint has been created, you would be able to perform real-time inference.

### C. Create a real time endpoint

In [ ]:
# Deploy the model to create an endpoint
instance_type = "ml.g5.2xlarge"
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

----------------------------------------------------------------

### D. Create an input payload

In [ ]:
input_sequence = "QYAPQTQSGRTSIVHLFEWRWVDIALECERYLGPKGFGGVQVSPPNENVVVTNPSRPWWERYQPVSYKLCTRSGNENEFRDMVTRCNNVGVRIYVDAVINHMCGSGAAAGTGTTCGSYCNPGSREFPAVPYSAWDFNDGKCKTASGGIESYNDPYQVRDCQLVGLLDLALEKDYVRSMIADYLNKLIDIGVAGFRIDASKHMWPGDIKAVLDKLHNLNTNWFPAGSRPFIFQEVIDLGGEAIKSSEYFGNGRVTEFKYGAKLGTVVRKWSGEKMSYLKNWGEGWGFMPSDRALVFVDNHDNQRGHGAGGSSILTFWDARLYKVAVGFMLAHPYGFTRVMSSYRWARNFVNGEDVNDWIGPPNNNGVIKEVTINADTTCGNDWVCEHRWREIRNMVWFRNVVDGEPFANWWDNGSNQVAFGRGNRGFIVFNNDDWQLSSTLQTGLPGGTYCDVISGDKVGNSCTGIKVYVSSDGTAQFSISNSAEDPFIAIHAESKL"

In [ ]:
import json
payload = json.dumps({"model": "esm3-sm-open-v1", "sequence": input_sequence, "sequence_logprobs": True})

In [ ]:
print(payload)

### E. Perform real-time inference

In [ ]:
# Invoke the endpoint

predictor.predict(
    ContentType='application/json',
    Body=payload
)

# Read and parse the response
result = json.loads(response['Body'].read().decode())

In [ ]:
from pprint import pprint

pprint(result)

## 4. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
# Delete the endpoint
try:
    response = sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"Endpoint '{endpoint_name}' deletion initiated.")
except sagemaker_client.exceptions.ClientError as e:
    print(f"Error deleting endpoint: {e}")

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

